# Data Analysis Using Numpy and pandas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as mlt
import mysql.connector
from scipy import stats

In [2]:
connection={
    "host":"localhost",
    "user":"root",
    "password":"12345678",
    "database":"Ola_data",
    "port":"3306"
}


In [ ]:
df=pd.read_csv("../data_generation/kola_bookings.csv")

### 🧪 A. Data Validation & Quality Checks (Pandas)


#### 1. What is the total number of records in the dataset?

In [4]:
print(f"Dataset contains {len(df)} records")

Dataset contains 100000 records


#### 2. What percentage of values are missing in each column?

In [5]:
missing_value_data=df.isna().mean()*100
for i,v in missing_value_data.items():
    print(f""" column "{i}" has {round(v,2)}% missing values""")

 column "Date" has 0.0% missing values
 column "Time" has 0.0% missing values
 column "Booking_ID" has 0.0% missing values
 column "Booking_Status" has 0.0% missing values
 column "Customer_ID" has 0.0% missing values
 column "Vehicle_Type" has 0.0% missing values
 column "Pickup_Location" has 0.0% missing values
 column "Drop_Location" has 0.0% missing values
 column "Avg_VTAT" has 32.52% missing values
 column "Avg_CTAT" has 32.52% missing values
 column "Cancelled_Rides_by_Customer" has 92.37% missing values
 column "Reason_for_cancelling_by_Customer" has 92.37% missing values
 column "Cancelled_Rides_by_Driver" has 80.44% missing values
 column "Driver_Cancel_Reason" has 80.44% missing values
 column "Incomplete_Rides" has 94.67% missing values
 column "Incomplete_Rides_Reason" has 94.67% missing values
 column "Booking_Value" has 32.52% missing values
 column "Payment_Method" has 32.52% missing values
 column "Ride_Distance" has 32.52% missing values
 column "Driver_Ratings" has 3

#### 3. Are there any duplicate Booking_IDs?

In [6]:
df["Booking_ID"].duplicated().value_counts()

Booking_ID
False    99521
True       479
Name: count, dtype: int64

#### 4. Do booking status percentages in Pandas match the SQL results?

In [236]:
status = df['Booking_Status'].value_counts() / df['Booking_Status'].count() * 100
print("="*20," booking status percentages Using Pandas","="*20)
for i, v in status.items():
    print(f"\"{i}\" category has {round(v, 2)}% out of 100")
print("="*90)
try:
    with mysql.connector.connect(**connection) as conn:
        with conn.cursor() as cur:
            query = """
                    SELECT 
                        booking_status,
                        round(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM bookings),2) AS booking_status_Percentage
                    FROM bookings
                    GROUP BY booking_Status
                    order by booking_status_Percentage desc;
            """
            cur.execute(query)

            data = cur.fetchall()
            columns = [col[0] for col in cur.description]
            df = pd.DataFrame(data, columns=columns)

            print("="*20, " booking status percentages", "="*20)
            print(df)
            print("="*84)

except Exception as e:
    print("Database error:", e)

====================  booking status percentages Using Pandas ====================
"Successfull" category has 67.48% out of 100
"Cancelled by Driver" category has 19.56% out of 100
"Cancelled by Customer" category has 7.63% out of 100
"Incomplete" category has 5.33% out of 100
====================  booking status percentages ====================
          booking_status booking_status_Percentage
0            Successfull                     67.48
1    Cancelled by Driver                     19.57
2  Cancelled by Customer                      7.63
3             Incomplete                      5.33


#### 5. Does total revenue calculated in Pandas match the SQL revenue?

In [238]:
print(f"Total revenue Generated = {int(df['Booking_Value'].sum())}")# its match the sql calaculation too
try:
    with mysql.connector.connect(**connection) as conn:
        with conn.cursor() as cur:
            query = """
                     select 
                        sum(Booking_Value) as total_Revenue
                     from 
                        bookings;
            """
            cur.execute(query)

            data = cur.fetchall()
            columns = [col[0] for col in cur.description]
            df = pd.DataFrame(data, columns=columns)

            print("="*20, "Table Structure: bookings", "="*20)
            print(df)
            print("="*84)

except Exception as e:
    print("Database error:", e)

Total revenue Generated = 30275567
==================== Table Structure: bookings ====================
  total_Revenue
0      30275567


#### 6. Are there any negative or zero values in Ride_Distance or Booking_Value?

In [7]:
counts = {
    "Ride_Distance": (df['Ride_Distance'] <= 0).sum(),
    "Booking_Value": (df['Booking_Value'] <= 0).sum()
}
print(counts,"\n no there isnt any negative or zero values in Ride_Distance or Booking_Value")

{'Ride_Distance': np.int64(0), 'Booking_Value': np.int64(0)} 
 no there isnt any negative or zero values in Ride_Distance or Booking_Value


#### 7. Are ratings (Driver_Ratings, Customer_Rating) within the valid range (1–5)?

In [8]:
invalid_count = ((df['Driver_Ratings'] < 1) | (df['Driver_Ratings'] > 5)).sum()
print("Driver_Ratings are within the valid range (1–5)" if invalid_count==0 else "Driver_Ratings are not within the valid range (1–5)" )
invalid_count = ((df['Customer_Rating'] < 1) | (df['Customer_Rating'] > 5)).sum()
print("Customer_Rating are within the valid range (1–5)" if invalid_count==0 else "Customer_Rating are not within the valid range (1–5)" )


Driver_Ratings are within the valid range (1–5)
Customer_Rating are within the valid range (1–5)


#### 8. Are there successful bookings with missing ratings or booking values?

In [9]:
successful_df = df[df['Booking_Status'] == "Successfull"]

missing_rating_percentage = successful_df['Driver_Ratings'].isna().mean() * 100
missing_booking_value = successful_df['Booking_Value'].isna().mean() * 100
print(f"Successful booking has {round(missing_rating_percentage, 2)} missing rating")
print(f"Successful booking has {round(missing_booking_value, 2)} booking values")

Successful booking has 0.0 missing rating
Successful booking has 0.0 booking values


#### 9. Are there cancelled bookings that incorrectly contain ride distance or ratings?

In [10]:
# Filter cancelled bookings
cancelled_df = df[df['Booking_Status'].isin(
    ['Cancelled by Driver', 'Cancelled by Customer']
)]

# Cancelled bookings WITH ride distance (should usually be NaN)
cancelled_with_distance = cancelled_df[cancelled_df['Ride_Distance'].notna()]

# Cancelled bookings WITH ratings (should usually be NaN)
cancelled_with_ratings = cancelled_df[
    cancelled_df['Driver_Ratings'].notna() |
    cancelled_df['Customer_Rating'].notna()
]

# Percentages
distance_pct = len(cancelled_with_distance) / len(cancelled_df) * 100
ratings_pct = len(cancelled_with_ratings) / len(cancelled_df) * 100

print(f"{distance_pct:.2f}% of cancelled bookings contain ride distance")
print(f"{ratings_pct:.2f}% of cancelled bookings contain ratings")


0.00% of cancelled bookings contain ride distance
0.00% of cancelled bookings contain ratings


#### 10. What columns contain the highest proportion of NULL values?

In [11]:
missing_value_data = df.isna().mean() * 100

for col, v in missing_value_data.sort_values(ascending=False).head(2).items():
    print(f"{col}: has maximum proportion of missing values i.e.  {v:.2f}%")


Incomplete_Rides_Reason: has maximum proportion of missing values i.e.  94.67%
Incomplete_Rides: has maximum proportion of missing values i.e.  94.67%


### 🧩 B. Feature Engineering — Columns to Create (Pandas)

#### 1. Can we create a binary column to identify successful bookings?

In [12]:
df['is_successful'] = (df['Booking_Status'] == 'Successfull').astype(int)
successful_df = df.groupby('Vehicle_Type')['is_successful'].mean()
print("=" * 20, "successful Booking Rate by Vehicle Type (%)", "=" * 20)
print("1. Can we create a binary column to identify successful bookings?\n->Yes We Can")
print(round(successful_df*100,2))
print("=" * 80)

==================== successful Booking Rate by Vehicle Type (%) ====================
1. Can we create a binary column to identify successful bookings?
->Yes We Can
Vehicle_Type
Auto           67.55
Bike           67.27
Mini           67.72
Prime Plus     67.77
Prime SUV      67.12
Prime Sedan    67.70
eBike          67.22
Name: is_successful, dtype: float64


#### 2. Can we create a binary column to identify cancelled bookings?

In [13]:
df['is_unsuccessful'] = df['Booking_Status'].isin(
    ['Cancelled by Driver', 'Cancelled by Customer']
).astype(int)

print("Unsuccessful bookings:", df['is_unsuccessful'].sum())

unsuccessful_rate = (
    df.groupby('Vehicle_Type')['is_unsuccessful']
      .mean()
      .mul(100)
      .round(2)
)

print("=" * 20, "Unsuccessful Booking Rate by Vehicle Type (%)", "=" * 20)
print("2. Can we create a binary column to identify cancelled bookings?\n->Yes We Can")

print(unsuccessful_rate)
print("=" * 80)


Unsuccessful bookings: 27195
==================== Unsuccessful Booking Rate by Vehicle Type (%) ====================
2. Can we create a binary column to identify cancelled bookings?
->Yes We Can
Vehicle_Type
Auto           27.22
Bike           27.20
Mini           27.08
Prime Plus     26.95
Prime SUV      27.54
Prime Sedan    27.08
eBike          27.30
Name: is_unsuccessful, dtype: float64


#### 3. Can we create a column that identifies weekend vs weekday rides?

In [14]:
df['Date'] = pd.to_datetime(df['Date'])
df['is_weekend'] = df['Date'].dt.dayofweek.isin([5, 6]).astype(int)
df['is_weekday'] = (df['is_weekend'] == 0).astype(int)
print("=" * 20, "column that identifies weekend vs weekday rides", "=" * 20)
print(df[['Date', 'is_weekend','is_weekday']].head())

==================== column that identifies weekend vs weekday rides ====================
        Date  is_weekend  is_weekday
0 2025-11-26           0           1
1 2025-11-19           0           1
2 2025-11-12           0           1
3 2025-11-12           0           1
4 2025-11-29           1           0


#### 4. Can we bucket Booking_Value into price segments (<500, 500–1000, >1000)?

In [15]:
df['lt_500'] = df['Booking_Value'].apply(
    lambda x: 1 if pd.notna(x) and x < 500 else 0
)
print("="*20,"bucket Booking_Value into price segments <500","="*20)
print(df['lt_500'].head())

print("="*20,"bucket Booking_Value into price segments 500-1000","="*20)
df['bt_500-1000'] = df['Booking_Value'].apply(
    lambda x: 1 if pd.notna(x) and (x >500 and x<1000) else 0
)
print(df['bt_500-1000'].head())

print("="*20,"bucket Booking_Value into price segments >1000","="*20)
df['gt_1000'] = df['Booking_Value'].apply(
    lambda x: 1 if pd.notna(x) and x > 1000 else 0
)
print(df['gt_1000'].head(10))



==================== bucket Booking_Value into price segments <500 ====================
0    0
1    1
2    0
3    0
4    0
Name: lt_500, dtype: int64
==================== bucket Booking_Value into price segments 500-1000 ====================
0    0
1    0
2    1
3    0
4    1
Name: bt_500-1000, dtype: int64
==================== bucket Booking_Value into price segments >1000 ====================
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: gt_1000, dtype: int64


#### 5. Can we create a cancellation flag combining customer and driver cancellations?

In [16]:
df['cancellation_flag'] = df['Booking_Status'].isin(
    ['Cancelled by Driver', 'Cancelled by Customer']
).astype(int)
print("="*20," cancellation flag combining customer and driver cancellations","="*20)
print(df['cancellation_flag'].head())
print(df['cancellation_flag'].value_counts())
print("="*105)


====================  cancellation flag combining customer and driver cancellations ====================
0    1
1    0
2    0
3    1
4    0
Name: cancellation_flag, dtype: int64
cancellation_flag
0    72805
1    27195
Name: count, dtype: int64


#### 6. Can we calculate total cancellations per customer?

In [17]:
cancellations_per_customer = (
    df.groupby('Customer_ID')['cancellation_flag']
      .sum()
      .reset_index(name='total_cancellations')
)
print("="*20,"calculate total cancellations per customer","="*20)

print(cancellations_per_customer)
print("="*85)

==================== calculate total cancellations per customer ====================
      Customer_ID  total_cancellations
0       CUST10000                    0
1       CUST10001                    1
2       CUST10002                    1
3       CUST10003                    0
4       CUST10004                    0
...           ...                  ...
60485   CUST99992                    0
60486   CUST99993                    2
60487   CUST99995                    1
60488   CUST99998                    0
60489   CUST99999                    0

[60490 rows x 2 columns]


#### 7. Can we calculate total rides per customer?

In [18]:
total_rides_per_customer = (
    df['Customer_ID']
      .value_counts()
      .reset_index(name='Total_Rides_Per_Customer')
      .rename(columns={'index': 'Customer_ID'})
)


print("="*20, "Total rides per customer", "="*20)
print(total_rides_per_customer)
print("="*85)


==================== Total rides per customer ====================
      Customer_ID  Total_Rides_Per_Customer
0       CUST53494                         8
1       CUST76276                         7
2       CUST91735                         7
3       CUST54071                         7
4       CUST56330                         7
...           ...                       ...
60485   CUST82437                         1
60486   CUST35499                         1
60487   CUST51477                         1
60488   CUST76583                         1
60489   CUST27380                         1

[60490 rows x 2 columns]


#### 8. Can we derive a customer-level cancellation rate?

In [19]:
# 1. Customer-level cancellation rate
customer_cancellation_rate = (
    df.groupby('Customer_ID')['cancellation_flag']
      .mean()
      .reset_index(name='cancellation_rate')
)

# 2. Convert to percentage
customer_cancellation_rate['cancellation_rate'] = (
    customer_cancellation_rate['cancellation_rate'] * 100
).round(2)

# 3. Sort by highest cancellation rate
customer_cancellation_rate = customer_cancellation_rate.sort_values(
    by='cancellation_rate', ascending=False
)

# 4. Display
print("="*20, "Customer-level Cancellation Rate (%)", "="*20)
print(customer_cancellation_rate.head())
print("="*85)
customer_stats = (
    df.groupby('Customer_ID')
      .agg(
          total_rides=('Booking_ID', 'count'),
          cancellation_rate=('cancellation_flag', 'mean')
      )
      .reset_index()
)

# Filter customers with <= 8 rides
customer_stats = customer_stats[
    customer_stats['total_rides'] <= 8
]

# Convert to percentage
customer_stats['cancellation_rate'] = (
    customer_stats['cancellation_rate'] * 100
).round(2)

# Sort by highest cancellation rate
customer_stats = customer_stats.sort_values(
    by='cancellation_rate', ascending=False
)

print(customer_stats)


==================== Customer-level Cancellation Rate (%) ====================
      Customer_ID  cancellation_rate
8       CUST10012              100.0
60472   CUST99973              100.0
60479   CUST99986              100.0
19511   CUST39054              100.0
19556   CUST39127              100.0
      Customer_ID  total_rides  cancellation_rate
8       CUST10012            1              100.0
60472   CUST99973            1              100.0
60479   CUST99986            1              100.0
19511   CUST39054            1              100.0
19556   CUST39127            1              100.0
...           ...          ...                ...
60454   CUST99953            1                0.0
60455   CUST99954            1                0.0
60458   CUST99957            1                0.0
17704   CUST36377            1                0.0
60471   CUST99972            1                0.0

[60490 rows x 3 columns]


#### 9. Can we calculate the difference between customer and driver ratings?

In [20]:
# Ensure ratings are numeric (safe guard)
df['Customer_Rating'] = pd.to_numeric(df['Customer_Rating'], errors='coerce')
df['Driver_Ratings'] = pd.to_numeric(df['Driver_Ratings'], errors='coerce')

# 1. Create rating difference column (Customer - Driver)
df['rating_difference'] = (
    df['Customer_Rating'] - df['Driver_Ratings']
)

# 2. Remove rows where either rating is missing (optional but recommended)
rating_df = df.dropna(subset=['Customer_Rating', 'Driver_Ratings'])

# 3. Basic statistics
print("="*20, "Rating Difference Summary", "="*20)
print(round(rating_df['rating_difference'].describe(),2))
print("="*80)

==================== Rating Difference Summary ====================
count    67478.00
mean        -0.00
std          0.61
min         -1.50
25%         -0.40
50%          0.00
75%          0.40
max          1.50
Name: rating_difference, dtype: float64


#### 10. Can we create a high-value booking indicator?

In [21]:
# Ensure Booking_Value is numeric
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

# 1. Create high-value booking indicator (IN-PLACE)
df['high_value_booking'] = (df['Booking_Value'] >= 500).astype(int)

# 2. Sanity check
print(df['high_value_booking'].value_counts())

# 3. Percentage of high-value bookings
high_value_percentage = round(
    df['high_value_booking'].mean() * 100, 2
)

print("="*20, "High-Value Booking Analysis", "="*20)
print(f"High-value bookings: {high_value_percentage}% of total bookings")
print("="*80)

high_value_booking
0    79625
1    20375
Name: count, dtype: int64
==================== High-Value Booking Analysis ====================
High-value bookings: 20.38% of total bookings


#### 11. Can we identify repeat customers?

In [22]:
# 1. Count total rides per customer
customer_rides = (
    df.groupby('Customer_ID')['Booking_ID']
      .count()
      .reset_index(name='total_rides')
)

# 2. Create repeat customer flag (IN-PLACE on new df)
customer_rides['is_repeat_customer'] = (
    customer_rides['total_rides'] > 1
).astype(int)

# 3. View results
print("="*20, "Repeat Customer Identification", "="*20)
print(customer_rides.head())
print("="*85)


==================== Repeat Customer Identification ====================
  Customer_ID  total_rides  is_repeat_customer
0   CUST10000            1                   0
1   CUST10001            2                   1
2   CUST10002            2                   1
3   CUST10003            1                   0
4   CUST10004            1                   0


#### 12. Can we tag customers as high-risk based on cancellation behavior?

In [23]:
# 1. Ensure cancellation_flag exists
df['cancellation_flag'] = df['Booking_Status'].isin(
    ['Cancelled by Driver', 'Cancelled by Customer']
).astype(int)

# 2. Build customer-level stats
customer_stats = (
    df.groupby('Customer_ID')
      .agg(
          total_rides=('Booking_ID', 'count'),
          cancellation_rate=('cancellation_flag', 'mean')
      )
      .reset_index()
)

# 3. Convert cancellation rate to percentage
customer_stats['cancellation_rate'] = (
    customer_stats['cancellation_rate'] * 100
).round(2)

# 4. Tag high-risk customers
customer_stats['high_risk_customer'] = (
    (customer_stats['total_rides'] >= 3) &
    (customer_stats['cancellation_rate'] >= 50)
).astype(int)

# 5. View high-risk customers
high_risk_customers = customer_stats[
    customer_stats['high_risk_customer'] == 1
]

print("="*20, "High-Risk Customers", "="*20)
print(high_risk_customers.head())
print("="*90)


==================== High-Risk Customers ====================
    Customer_ID  total_rides  cancellation_rate  high_risk_customer
7     CUST10011            3              66.67                   1
117   CUST10162            3             100.00                   1
118   CUST10163            3              66.67                   1
143   CUST10198            3              66.67                   1
178   CUST10246            4              50.00                   1


### 📊 C. Exploratory Analysis (Pandas)

#### 1. What is the distribution of booking values?

In [24]:
# Convert to numeric safely
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

# Distribution summary
print("="*20, "The distribution of booking values", "="*20)
print(df['Booking_Value'].describe().round(2))
print("="*100)

# Skewness
skewness = df['Booking_Value'].skew()
print(f"Skewness of Booking_Value: {round(skewness, 2)}")


==================== The distribution of booking values ====================
count    67478.00
mean       448.67
std        275.14
min        100.00
25%        243.00
50%        387.00
75%        594.00
max       1799.00
Name: Booking_Value, dtype: float64
Skewness of Booking_Value: 1.31


#### 2. What is the distribution of ride distances?

In [25]:
# Convert to numeric safely
df['Ride_Distance'] = pd.to_numeric(df['Ride_Distance'], errors='coerce')

# Distribution summary
print("="*20, "distribution of ride distances", "="*20)
print(df['Ride_Distance'].describe().round(2))
print("="*100)

# Skewness
skewness = df['Ride_Distance'].skew()
print(f"Skewness of ride distances: {round(skewness, 2)}")


==================== distribution of ride distances ====================
count    67478.00
mean        17.93
std          9.83
min          1.00
25%          9.36
50%         17.89
75%         26.49
max         35.00
Name: Ride_Distance, dtype: float64
Skewness of ride distances: 0.01


#### 3. How do booking values vary by vehicle type?


In [26]:
df['Booking_Value']=pd.to_numeric(df['Booking_Value'], errors='coerce')
vehicle_summary=(df.groupby('Vehicle_Type')['Booking_Value']
             .describe()
             .round(2))
print(vehicle_summary)

for vehicle, stats in vehicle_summary.iterrows():
    mean_value = stats['mean']
    median_value = stats['50%']
    print("="*20, f"{vehicle} Booking Value Analysis", "="*20)
    print(f"Vehicle Type: {vehicle}")
    print(f"Average Booking Value : {mean_value}")
    print(f"Typical Booking Value : {median_value}")
    if mean_value > median_value:
        print("Most rides cost a normal amount, but a few expensive rides increase the average price.")
    elif mean_value < median_value:
        print("Most rides are expensive, but some cheaper rides bring the average down.")
    else:
        print("Most rides cost around the same amount. Prices are consistent.")


               count    mean     std    min     25%    50%    75%     max
Vehicle_Type                                                             
Auto          9768.0  452.00  279.54  100.0  244.00  387.0  601.0  1792.0
Bike          9510.0  448.95  270.86  100.0  244.00  390.0  597.0  1796.0
Mini          9740.0  450.04  277.27  100.0  244.00  385.5  598.0  1798.0
Prime Plus    9618.0  449.08  269.84  100.0  246.25  387.0  600.0  1795.0
Prime SUV     9514.0  447.96  274.68  100.0  243.00  387.0  590.0  1799.0
Prime Sedan   9669.0  444.73  277.86  100.0  240.00  383.0  575.0  1797.0
eBike         9659.0  447.91  275.66  100.0  240.00  386.0  594.0  1793.0
==================== Auto Booking Value Analysis ====================
Vehicle Type: Auto
Average Booking Value : 452.0
Typical Booking Value : 387.0
Most rides cost a normal amount, but a few expensive rides increase the average price.
==================== Bike Booking Value Analysis ====================
Vehicle Type: Bike
Average B

#### 4. How do ride distances vary by vehicle type?

In [27]:
df['Ride_Distance']=pd.to_numeric(df['Ride_Distance'],errors="coerce")
vehicle_distance_summary=(df.groupby('Vehicle_Type')['Ride_Distance']
                                    .describe()
                                    .round(2))
print(vehicle_distance_summary)
for vehicle,stats in vehicle_distance_summary.iterrows():
    mean_value=stats['mean']
    median_value=stats['50%']
    print("="*20,f"{vehicle} Ride Distance Analysis","="*20)
    print(f"Average Ride Distance:{mean_value}")
    print(f"Typical Ride Distance:{median_value}")
    if mean_value > median_value:
        print("Meaning: Most rides are shorter, but a few long-distance rides increase the average distance.")
    elif mean_value < median_value:
        print("Meaning: Most rides are longer, but a few long-distance rides reduce the average distance.")
    else:
        print("Meaning: Ride Distance are consistent for this Vehicle Type ")

               count   mean   std   min   25%    50%    75%    max
Vehicle_Type                                                      
Auto          9768.0  18.11  9.79  1.00  9.72  18.12  26.59  35.00
Bike          9510.0  17.87  9.82  1.00  9.32  17.79  26.42  35.00
Mini          9740.0  17.91  9.91  1.01  9.18  17.96  26.58  35.00
Prime Plus    9618.0  17.85  9.82  1.00  9.34  17.72  26.38  34.99
Prime SUV     9514.0  17.85  9.86  1.00  9.18  17.84  26.51  35.00
Prime Sedan   9669.0  18.04  9.82  1.01  9.50  18.01  26.61  35.00
eBike         9659.0  17.90  9.80  1.00  9.39  17.75  26.26  34.99
==================== Auto Ride Distance Analysis ====================
Average Ride Distance:18.11
Typical Ride Distance:18.12
Meaning: Most rides are longer, but a few long-distance rides reduce the average distance.
==================== Bike Ride Distance Analysis ====================
Average Ride Distance:17.87
Typical Ride Distance:17.79
Meaning: Most rides are shorter, but a few long-distan

#### 5. How does customer rating differ between successful and cancelled rides?

In [28]:
df['Customer_Rating']=pd.to_numeric(df['Customer_Rating'],errors='coerce')
customer_rating=(df.groupby('Booking_Status')['Customer_Rating']
                .describe()
                .round(2))
print("="*20, "Customer Rating by Booking Status", "="*20)
print(customer_rating)
print("="*75,"\n")
print("="*58,"Final Verdict:","="*58)
print("1. Successful Rides:\n - Customer ratings are available. \n - Average customer rating is around 4.25, indicating generally high satisfaction after completed trips.")
print("2. Cancelled / Incomplete Rides:\n - Customer ratings are not available.\n - This is because the ride did not get completed, so customers could not provide a rating.")
print("3. Conclusion:\n - Customer satisfaction can only be measured for successful rides. \n - Cancelled and incomplete rides must be analyzed using other factors such as cancellation reasons, timing, or customer behavior.")
print("="*133)

==================== Customer Rating by Booking Status ====================
                         count  mean   std  min  25%  50%  75%  max
Booking_Status                                                     
Cancelled by Customer      0.0   NaN   NaN  NaN  NaN  NaN  NaN  NaN
Cancelled by Driver        0.0   NaN   NaN  NaN  NaN  NaN  NaN  NaN
Incomplete                 0.0   NaN   NaN  NaN  NaN  NaN  NaN  NaN
Successfull            67478.0  4.25  0.44  3.5  3.9  4.2  4.6  5.0

========================================================== Final Verdict: ==========================================================
1. Successful Rides:
 - Customer ratings are available. 
 - Average customer rating is around 4.25, indicating generally high satisfaction after completed trips.
2. Cancelled / Incomplete Rides:
 - Customer ratings are not available.
 - This is because the ride did not get completed, so customers could not provide a rating.
3. Conclusion:
 - Customer satisfaction can only be meas

#### 6. How does driver rating differ across vehicle types?

In [29]:
df['Driver_Ratings'] = pd.to_numeric(df['Driver_Ratings'], errors='coerce')

# Step 2: Group and describe
driver_rating_summary = (
    df.groupby('Vehicle_Type')['Driver_Ratings']
      .describe()
      .round(2)
)
print("="*20,"Driver Rating Summary","="*20)
print(driver_rating_summary)
print("="*63)
for vehicle, stats in driver_rating_summary.iterrows():
    mean_value = stats['mean']
    median_value = stats['50%']

    print("="*31, f"{vehicle} Driver Rating Analysis", "="*31)
    print(f"Average Driver Rating : {round(mean_value, 2)}")
    print(f"Typical Driver Rating : {round(median_value, 2)}")

    if mean_value > median_value:
        print(
            "Meaning: Most drivers receive good ratings, but a few very high ratings increase the average."
        )
    elif mean_value < median_value:
        print(
            "Meaning: Most drivers receive high ratings, but a few lower ratings slightly reduce the average."
        )
    else:
        print(
            "Meaning: Driver ratings are consistent across most rides."
        )

    print("-" * 92)


==================== Driver Rating Summary ====================
               count  mean   std  min  25%  50%  75%  max
Vehicle_Type                                             
Auto          9768.0  4.25  0.43  3.5  3.9  4.3  4.6  5.0
Bike          9510.0  4.24  0.44  3.5  3.9  4.2  4.6  5.0
Mini          9740.0  4.25  0.44  3.5  3.9  4.3  4.6  5.0
Prime Plus    9618.0  4.25  0.43  3.5  3.9  4.3  4.6  5.0
Prime SUV     9514.0  4.25  0.44  3.5  3.9  4.3  4.6  5.0
Prime Sedan   9669.0  4.25  0.43  3.5  3.9  4.3  4.6  5.0
eBike         9659.0  4.25  0.43  3.5  3.9  4.2  4.6  5.0
=============================== Auto Driver Rating Analysis ===============================
Average Driver Rating : 4.25
Typical Driver Rating : 4.3
Meaning: Most drivers receive high ratings, but a few lower ratings slightly reduce the average.
--------------------------------------------------------------------------------------------
=============================== Bike Driver Rating Analysis ===============

#### 7. Are higher booking values associated with longer ride distances?

In [30]:
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')
df['Ride_Distance'] = pd.to_numeric(df['Ride_Distance'], errors='coerce')

clean_df = df.dropna(subset=['Booking_Value', 'Ride_Distance'])

correlation = round(
    clean_df['Booking_Value'].corr(clean_df['Ride_Distance']), 4
)

print("="*20, "Booking Value vs Ride Distance Analysis", "="*20)
print(f"Correlation coefficient: {correlation}")

if correlation > 0.5:
    print("Meaning: Longer rides usually cost significantly more.")
elif correlation > 0.3:
    print("Meaning: Booking value generally increases as ride distance increases.")
elif correlation > 0:
    print("Meaning: Distance has a small impact on booking value.")
else:
    print("Meaning: Ride distance does not meaningfully affect booking value.")

print("="*80)


==================== Booking Value vs Ride Distance Analysis ====================
Correlation coefficient: 0.0027
Meaning: Distance has a small impact on booking value.


#### 8. How does cancellation rate vary by vehicle type?

In [31]:
df['cancellation_flag'] = pd.to_numeric(df['cancellation_flag'], errors='coerce')

vehicle_cancellation_rate_summary = (
    df.groupby('Vehicle_Type')['cancellation_flag']
      .describe()
      .round(2)
)

print(vehicle_cancellation_rate_summary)

for vehicle, stats in vehicle_cancellation_rate_summary.iterrows():
    mean_value = stats['mean']
    median_value = stats['50%']

    print("="*45, f"{vehicle} Cancellation Analysis", "="*45)
    print(f"Average Cancellation Rate : {round(mean_value * 100, 2)}%")
    print(f"Typical Outcome           : {'Cancelled' if median_value == 1 else 'Not Cancelled'}")

    if mean_value > median_value:
        print(
            "Meaning: Most rides are completed, but a noticeable number of cancellations increase the overall cancellation rate."
        )
    elif mean_value < median_value:
        print(
            "Meaning: Cancellations are frequent and affect most rides."
        )
    else:
        print(
            "Meaning: Cancellation behavior is consistent for this vehicle type."
        )

    print("-" * 120)


                count  mean   std  min  25%  50%  75%  max
Vehicle_Type                                              
Auto          14461.0  0.27  0.45  0.0  0.0  0.0  1.0  1.0
Bike          14138.0  0.27  0.44  0.0  0.0  0.0  1.0  1.0
Mini          14382.0  0.27  0.44  0.0  0.0  0.0  1.0  1.0
Prime Plus    14193.0  0.27  0.44  0.0  0.0  0.0  1.0  1.0
Prime SUV     14174.0  0.28  0.45  0.0  0.0  0.0  1.0  1.0
Prime Sedan   14282.0  0.27  0.44  0.0  0.0  0.0  1.0  1.0
eBike         14370.0  0.27  0.45  0.0  0.0  0.0  1.0  1.0
============================================= Auto Cancellation Analysis =============================================
Average Cancellation Rate : 27.0%
Typical Outcome           : Not Cancelled
Meaning: Most rides are completed, but a noticeable number of cancellations increase the overall cancellation rate.
------------------------------------------------------------------------------------------------------------------------
=====================================

#### 9. How does cancellation rate differ between weekdays and weekends?

In [32]:
# cancellation_flag:
# 1 = Cancelled
# 0 = Not Cancelled

# Step 1: Ensure numeric
df['cancellation_flag'] = pd.to_numeric(df['cancellation_flag'], errors='coerce')

# Step 2: Calculate cancellation rate for weekday vs weekend
cancellation_rate = (
    df.groupby(['is_weekend','is_weekday'])['cancellation_flag']
      .mean()
      .reset_index(name='cancellation_rate')
)

# Step 3: Convert to percentage
cancellation_rate['cancellation_rate'] = (
    cancellation_rate['cancellation_rate'] * 100
).round(2)

print("="*20, "Cancellation Rate: Weekday vs Weekend", "="*20)
print(cancellation_rate)
print("="*80)


# Extract values
weekday_rate = cancellation_rate.loc[
    cancellation_rate['is_weekend'] == 0, 'cancellation_rate'
].values[0]

weekend_rate = cancellation_rate.loc[
    cancellation_rate['is_weekend'] == 1, 'cancellation_rate'
].values[0]

if weekend_rate > weekday_rate:
    print(
        f"Meaning: Cancellations are slightly higher on weekends "
        f"({weekend_rate}%) compared to weekdays ({weekday_rate}%), "
        f"but the difference is very small."
    )
elif weekend_rate < weekday_rate:
    print(
        f"Meaning: Cancellations are slightly higher on weekdays "
        f"({weekday_rate}%) compared to weekends ({weekend_rate}%), "
        f"but the difference is very small."
    )
else:
    print(
        "Meaning: Cancellation rates are exactly the same on weekdays and weekends."
    )

print("="*80)


==================== Cancellation Rate: Weekday vs Weekend ====================
   is_weekend  is_weekday  cancellation_rate
0           0           1              27.14
1           1           0              27.30
Meaning: Cancellations are slightly higher on weekends (27.3%) compared to weekdays (27.14%), but the difference is very small.


#### 10. Which vehicle types generate the highest average revenue?

In [33]:
# Ensure numeric
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

# Average booking value by vehicle type
vehicle_revenue_summary = (
    df.groupby('Vehicle_Type')['Booking_Value']
      .mean()
      .round(2)
)


# Vehicle with highest average revenue
top_vehicle = vehicle_revenue_summary.idxmax()
top_value = vehicle_revenue_summary.max()

print("="*20, "Highest Average Revenue", "="*20)
print(f"Vehicle Type : {top_vehicle}")
print(f"Average Revenue : {top_value}")
print("-" * 80)


==================== Highest Average Revenue ====================
Vehicle Type : Auto
Average Revenue : 452.0
--------------------------------------------------------------------------------


#### 11. Which vehicle types have the lowest average ratings?

In [34]:
# Ensure numeric
df['Driver_Ratings'] = pd.to_numeric(df['Driver_Ratings'], errors='coerce')
df['Customer_Rating'] = pd.to_numeric(df['Customer_Rating'], errors='coerce')

# Average ratings by vehicle type
vehicle_rating_summary = (
    df.groupby('Vehicle_Type')[['Customer_Rating', 'Driver_Ratings']]
      .mean()
      .round(2)
)

print("="*20, "Average Ratings by Vehicle Type", "="*20)
print(vehicle_rating_summary)
print("-" * 80)

# Customer Rating analysis
highest_customer_vehicle = vehicle_rating_summary['Customer_Rating'].idxmax()
lowest_customer_vehicle  = vehicle_rating_summary['Customer_Rating'].idxmin()

# Driver Rating analysis
highest_driver_vehicle = vehicle_rating_summary['Driver_Ratings'].idxmax()
lowest_driver_vehicle  = vehicle_rating_summary['Driver_Ratings'].idxmin()

print("="*20, "Customer Rating Insight", "="*20)
print(f"Lowest average customer rating  : {lowest_customer_vehicle}")
print("-" * 80)

print("="*20, "Driver Rating Insight", "="*20)
print(f"Lowest average driver rating  : {lowest_driver_vehicle}")
print("-" * 80)


==================== Average Ratings by Vehicle Type ====================
              Customer_Rating  Driver_Ratings
Vehicle_Type                                 
Auto                     4.25            4.25
Bike                     4.25            4.24
Mini                     4.25            4.25
Prime Plus               4.24            4.25
Prime SUV                4.25            4.25
Prime Sedan              4.26            4.25
eBike                    4.25            4.25
--------------------------------------------------------------------------------
==================== Customer Rating Insight ====================
Lowest average customer rating  : Prime Plus
--------------------------------------------------------------------------------
==================== Driver Rating Insight ====================
Lowest average driver rating  : Bike
--------------------------------------------------------------------------------


#### 12. Which payment methods are associated with higher booking values?

In [35]:
# # Ensure numeric
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

# Average booking value by payment method 
payment_method_summary = (
    df.groupby('Payment_Method')['Booking_Value']
      .mean()
      .round(2)
)

# payment methods with higher booking values
top_payment_method = payment_method_summary.idxmax()
top_value = payment_method_summary.max()

print("="*20, " payment methods associated with higher booking values", "="*20)
print(payment_method_summary)
print(f"Payment Type : {top_payment_method}")
print(f"Highest Booking value : {top_value}")
print("=" * 95)

====================  payment methods associated with higher booking values ====================
Payment_Method
Cash           449.65
Credit Card    447.98
Debit Card     450.10
Ola Money      451.46
UPI            447.54
Name: Booking_Value, dtype: float64
Payment Type : Ola Money
Highest Booking value : 451.46


### 📐 D. NumPy-Based Statistical Analysis

#### 1. What is the correlation between ride distance and booking value?

In [42]:
df['Ride_Distance'] = pd.to_numeric(df['Ride_Distance'], errors='coerce')
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

clean_df = df.dropna(subset=['Booking_Value', 'Ride_Distance'])

correlation_between_RD_AND_BV = round(
    clean_df['Booking_Value'].corr(clean_df['Ride_Distance']), 6
)

print("="*20, "Correlation: Ride Distance vs Booking Value", "="*20)
print("Correlation coefficient:", correlation_between_RD_AND_BV)

abs_corr = abs(correlation_between_RD_AND_BV)

if abs_corr < 0.2:
    print("Meaning: The relationship is very weak. Ride distance has little impact on booking value.")
elif abs_corr < 0.4:
    print("Meaning: The relationship is weak. Distance has some influence on booking value.")
elif abs_corr < 0.6:
    print("Meaning: The relationship is moderate. Longer rides generally cost more.")
else:
    print("Meaning: The relationship is strong. Booking value increases clearly with distance.")




==================== Correlation: Ride Distance vs Booking Value ====================
Correlation coefficient: 0.002713
Meaning: The relationship is very weak. Ride distance has little impact on booking value.


#### 2. What is the correlation between driver rating and customer rating?

In [52]:
df['Driver_Ratings'] = pd.to_numeric(df['Driver_Ratings'], errors='coerce')
df['Customer_Rating'] = pd.to_numeric(df['Customer_Rating'], errors='coerce')

clean_df = df.dropna(subset=['Driver_Ratings', 'Customer_Rating'])

correlation_between_CR_AND_DR = round(
    clean_df['Driver_Ratings'].corr(clean_df['Customer_Rating']), 6
)

print("="*20, "Correlation: Driver Rating vs Customer Rating", "="*20)
print("Correlation coefficient:", correlation_between_CR_AND_DR)

abs_corr = abs(correlation_between_CR_AND_DR)

if abs_corr < 0.2:
    print(
        "Meaning: The relationship is very weak. Customer ratings have minimal influence on driver ratings."
    )
elif abs_corr < 0.4:
    print(
        "Meaning: The relationship is weak. Customer ratings have some influence on driver ratings."
    )
elif abs_corr < 0.6:
    print(
        "Meaning: The relationship is moderate. Customer ratings noticeably influence driver ratings."
    )
else:
    print(
        "Meaning: The relationship is strong. Customer ratings strongly influence driver ratings."
    )

print("="*80)


==================== Correlation: Driver Rating vs Customer Rating ====================
Correlation coefficient: 0.004735
Meaning: The relationship is very weak. Customer ratings have minimal influence on driver ratings.


#### 3. What is the correlation between booking value and customer rating?

In [53]:
df['Driver_Ratings'] = pd.to_numeric(df['Driver_Ratings'], errors='coerce')
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

clean_df = df.dropna(subset=['Driver_Ratings', 'Booking_Value'])

correlation_between_DR_AND_BV = round(
    clean_df['Driver_Ratings'].corr(clean_df['Booking_Value']), 6
)

print("="*20, "Correlation: Driver Rating vs Booking Value", "="*20)
print("Correlation coefficient:", correlation_between_DR_AND_BV)

abs_corr = abs(correlation_between_DR_AND_BV)

if abs_corr < 0.2:
    print(
        "Meaning: The relationship is very weak. Booking value has minimal influence on driver ratings."
    )
elif abs_corr < 0.4:
    print(
        "Meaning: The relationship is weak. Booking value has some influence on driver ratings."
    )
elif abs_corr < 0.6:
    print(
        "Meaning: The relationship is moderate. Booking value has a noticeable influence on driver ratings."
    )
else:
    print(
        "Meaning: The relationship is strong. Booking value strongly influences driver ratings."
    )

print("="*80)


==================== Correlation: Driver Rating vs Booking Value ====================
Correlation coefficient: 0.001287
Meaning: The relationship is very weak. Booking value has minimal influence on driver ratings.


#### 4. What are the 90th and 95th percentiles of booking value?

In [82]:
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

p95 = np.percentile(df['Booking_Value'].dropna(), 95)
p90 = np.percentile(df['Booking_Value'].dropna(), 90)

print("="*20, "90th and 95th Percentiles of Booking Value", "="*20)
print(f"95th Percentile (Top 5% bookings)  : {round(p95, 2)}")
print(f"90th Percentile (Top 10% bookings) : {round(p90, 2)}")
print("="*88)


==================== 90th and 95th Percentiles of Booking Value ====================
95th Percentile (Top 5% bookings)  : 946.0
90th Percentile (Top 10% bookings) : 855.0


#### 5. What are the 90th and 95th percentiles of ride distance?

In [94]:
df['Ride_Distance'] = pd.to_numeric(df['Ride_Distance'], errors='coerce')

p95 = np.percentile(df['Ride_Distance'].dropna(), 95)
p90 = np.percentile(df['Ride_Distance'].dropna(), 90)

print("="*20, "90th and 95th Percentiles of Ride_Distance", "="*20)
print(f"95th Percentile (Top 5% Ride_Distance)  : {round(p95, 2)}")
print(f"90th Percentile (Top 10% Ride_Distance) : {round(p90, 2)}")
print("="*88)


==================== 90th and 95th Percentiles of Ride_Distance ====================
95th Percentile (Top 5% Ride_Distance)  : 33.3
90th Percentile (Top 10% Ride_Distance) : 31.6


#### 6. Are there extreme outliers in booking value?

In [98]:
# formula for outlier
# q1= 25th percentile
# q2= 50th percentile i.e median 
# q3= 75th percentile
# IQR=Q3-Q1
# upper hinge=q3+1.5*iqr
# lower hinge=q1-1.5*iqr
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')
clean_df=df['Booking_Value'].dropna()
Q1=np.percentile(clean_df,25)
Q2=np.percentile(clean_df,50)
Q3=np.percentile(clean_df,75)
IQR=Q3-Q1
upper_hinge=Q3+1.5*IQR
lower_hinge=Q1-1.5*IQR
print("="*20, "extreme outliers in booking value", "="*20)
outliers = clean_df[
    (clean_df > upper_hinge) | (clean_df < lower_hinge)
]

print(f"Number of extreme outliers: {len(outliers)}")
if len(outliers) > 0:
    print(
        "Conclusion: Yes, extreme outliers exist in booking value. "
        "\nThese represent unusually high or low-priced rides, likely due to "
        "\nlong-distance trips, premium vehicles, or surge pricing."
    )
else:
    print(
        "Conclusion: No extreme outliers were found. "
        "\nBooking values are well distributed within the expected range."
    )
print("="*88)

==================== extreme outliers in booking value ====================
Number of extreme outliers: 1173
Conclusion: Yes, extreme outliers exist in booking value. 
These represent unusually high or low-priced rides, likely due to 
long-distance trips, premium vehicles, or surge pricing.


#### 7. Are there extreme outliers in ride distance?

In [99]:
# formula for outlier
# q1= 25th percentile
# q2= 50th percentile i.e median 
# q3= 75th percentile
# IQR=Q3-Q1
# upper hinge=q3+1.5*iqr
# lower hinge=q1-1.5*iqr
df['Ride_Distance'] = pd.to_numeric(df['Ride_Distance'], errors='coerce')
clean_df=df['Ride_Distance'].dropna()
Q1=np.percentile(clean_df,25)
Q2=np.percentile(clean_df,50)
Q3=np.percentile(clean_df,75)
IQR=Q3-Q1
upper_hinge=Q3+1.5*IQR
lower_hinge=Q1-1.5*IQR
print("="*20, "extreme outliers in Ride_Distance", "="*20)
outliers = clean_df[
    (clean_df > upper_hinge) | (clean_df < lower_hinge)
]

print(f"Number of extreme outliers: {len(outliers)}")
if len(outliers) > 0:
    print(
        "Conclusion: Yes, extreme outliers exist in Ride_Distance. "
        "\nThese represent unusually high or low-priced rides, likely due to "
        "\nlong-distance trips, premium vehicles, or surge pricing."
    )
else:
    print(
        "Conclusion: No extreme outliers were found. "
        "\nRide_Distance are well distributed within the expected range."
    )
print("="*88)

==================== extreme outliers in Ride_Distance ====================
Number of extreme outliers: 0
Conclusion: No extreme outliers were found. 
Ride_Distance are well distributed within the expected range.


#### 8. Can z-scores be used to identify abnormal customer cancellation behaviour?

In [106]:

from scipy.stats import zscore

# Ensure cancellation_flag is numeric
df['cancellation_flag'] = pd.to_numeric(df['cancellation_flag'], errors='coerce')

# 1) Customer-level aggregation
customer_cancellation = (
    df.groupby('Customer_ID')
      .agg(
          total_rides=('Booking_ID', 'count'),
          total_cancellations=('cancellation_flag', 'sum')
      )
      .reset_index()
)

# 2) Keep customers with sufficient activity (important)
customer_cancellation = customer_cancellation[
    customer_cancellation['total_rides'] >= 5
]

# 3) Cancellation rate per customer
customer_cancellation['cancellation_rate'] = (
    customer_cancellation['total_cancellations'] /
    customer_cancellation['total_rides']
)

# 4) Compute z-scores using SciPy
customer_cancellation['z_score'] = zscore(
    customer_cancellation['cancellation_rate'],
    nan_policy='omit'
)

# 5) Flag potentially risky customers
customer_cancellation['risk_flag'] = np.where(
    customer_cancellation['z_score'] >= 1.5,
    'Potential Risk',
    'Normal'
)

# 6) Display flagged customers
flagged_customers = customer_cancellation[
    customer_cancellation['risk_flag'] == 'Potential Risk'
].sort_values('z_score', ascending=False)

print("="*20, "Potentially Risky Customers (Z-score ≥ 1.5)", "="*20)
print(flagged_customers.head(10))
print("="*90)


==================== Potentially Risky Customers (Z-score ≥ 1.5) ====================
      Customer_ID  total_rides  total_cancellations  cancellation_rate  \
17150   CUST35572            5                    4           0.800000   
13780   CUST30528            5                    4           0.800000   
13151   CUST29602            5                    4           0.800000   
45535   CUST77880            5                    4           0.800000   
38121   CUST66795            5                    4           0.800000   
28330   CUST52218            5                    4           0.800000   
46603   CUST79469            5                    4           0.800000   
48281   CUST81964            5                    4           0.800000   
56578   CUST94199            5                    4           0.800000   
35935   CUST63488            6                    4           0.666667   

        z_score       risk_flag  
17150  2.776314  Potential Risk  
13780  2.776314  Potential Risk

#### 9. How many customers fall beyond the 95th percentile of cancellations?

In [109]:


# Ensure cancellation_flag is numeric
df['cancellation_flag'] = pd.to_numeric(df['cancellation_flag'], errors='coerce')

# 1) Customer-level aggregation
customer_cancellation = (
    df.groupby('Customer_ID')
      .agg(
          total_rides=('Booking_ID', 'count'),
          total_cancellations=('cancellation_flag', 'sum')
      )
      .reset_index()
)

# 2) Calculate cancellation rate
customer_cancellation['cancellation_rate'] = (
    customer_cancellation['total_cancellations'] /
    customer_cancellation['total_rides']
)

# 3) Find 95th percentile threshold
p95_threshold = np.percentile(
    customer_cancellation['cancellation_rate'].dropna(), 95
)

# 4) Customers beyond 95th percentile
high_cancellation_customers = customer_cancellation[
    customer_cancellation['cancellation_rate'] > p95_threshold
]

total_customers = customer_cancellation.shape[0]
high_count = high_cancellation_customers.shape[0]
percentage = round((high_count / total_customers) * 100, 2)

print("="*20, "Customers Beyond 95th Percentile of Cancellations", "="*20)
print(f"95th Percentile Cancellation Rate Threshold : {round(p95_threshold, 4)}")
print(f"Number of customers beyond threshold        : {high_count}")
print(f"Percentage of total customers               : {percentage}%")
print("-" * 90)

# 5) Data-driven conclusion
if high_count == 0:
    print(
        "Conclusion: No customers exceed the 95th percentile threshold. "
        "Cancellation behavior is evenly distributed with no extreme cases."
    )
elif percentage <= 5:
    print(
        "Conclusion: A small group of customers (around the top 5%) shows "
        "unusually high cancellation behavior, indicating potential risk users."
    )
else:
    print(
        "Conclusion: More than the expected proportion of customers exceed the "
        "95th percentile, suggesting widespread high cancellation behavior."
    )

print("="*90)


==================== Customers Beyond 95th Percentile of Cancellations ====================
95th Percentile Cancellation Rate Threshold : 1.0
Number of customers beyond threshold        : 0
Percentage of total customers               : 0.0%
------------------------------------------------------------------------------------------
Conclusion: No customers exceed the 95th percentile threshold. Cancellation behavior is evenly distributed with no extreme cases.


#### 10. Are high-value rides more likely to be successful?

In [108]:
import numpy as np
import pandas as pd

# Ensure numeric
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

# 1) Define high-value rides (top 10%)
high_value_threshold = np.percentile(
    df['Booking_Value'].dropna(), 90
)

df['is_high_value'] = (df['Booking_Value'] >= high_value_threshold).astype(int)

# 2) Create success flag
df['is_successful'] = (df['Booking_Status'] == 'Successfull').astype(int)

# 3) Compare success rates
success_rate_comparison = (
    df.groupby('is_high_value')['is_successful']
      .mean()
      .mul(100)
      .round(2)
      .reset_index(name='success_rate_percent')
)

print("="*20, "Success Rate: High vs Non-High Value Rides", "="*20)
print(f"High-value threshold (90th percentile): {round(high_value_threshold, 2)}")
print(success_rate_comparison)
print("="*90)

# 4) Interpretation
high_value_rate = success_rate_comparison.loc[
    success_rate_comparison['is_high_value'] == 1, 'success_rate_percent'
].values[0]

non_high_value_rate = success_rate_comparison.loc[
    success_rate_comparison['is_high_value'] == 0, 'success_rate_percent'
].values[0]

if high_value_rate > non_high_value_rate:
    print(
        "Conclusion: High-value rides are more likely to be successful than lower-value rides."
    )
elif high_value_rate < non_high_value_rate:
    print(
        "Conclusion: High-value rides are less likely to be successful than lower-value rides."
    )
else:
    print(
        "Conclusion: High-value and lower-value rides have similar success rates."
    )

print("="*90)


==================== Success Rate: High vs Non-High Value Rides ====================
High-value threshold (90th percentile): 855.0
   is_high_value  success_rate_percent
0              0                 65.12
1              1                100.00
Conclusion: High-value rides are more likely to be successful than lower-value rides.


### 👥 E. Customer Behavior & Risk Analysis

#### 1. Which customers have the highest cancellation rates?

In [18]:
cancellation_rate = (
    df.groupby('Customer_ID')
      .agg(
          total_rides=('Customer_ID', 'count'),
          cancelled_rides=('Cancelled_Rides_by_Customer', 'sum')
      )
)

cancellation_rate['cancellation_rate'] = (
    cancellation_rate['cancelled_rides'] / cancellation_rate['total_rides']
    .mul(0.01)
)

result = cancellation_rate.sort_values(
    by=['cancellation_rate'],
    ascending=False
)

print(result.head(10))


             total_rides  cancelled_rides  cancellation_rate
Customer_ID                                                 
CUST99941              1              1.0              100.0
CUST20135              1              1.0              100.0
CUST99948              1              1.0              100.0
CUST20227              1              1.0              100.0
CUST20060              1              1.0              100.0
CUST20064              1              1.0              100.0
CUST20072              1              1.0              100.0
CUST20073              1              1.0              100.0
CUST20052              1              1.0              100.0
CUST10012              1              1.0              100.0


#### 2. How many customers have cancelled more than once?

In [31]:
result_for_cancelled_more_than_one = (
    result[result['cancelled_rides'] > 1]
    .sort_values(by='cancelled_rides', ascending=False)
)

print(result_for_cancelled_more_than_one.count())


total_rides          316
cancelled_rides      316
cancellation_rate    316
dtype: int64


#### 3. What percentage of customers are repeat cancellers?

In [32]:
total_customers = cancellation_rate.shape[0]

repeat_cancellers = cancellation_rate[
    cancellation_rate['cancelled_rides'] > 1
].shape[0]

percentage_repeat_cancellers = (repeat_cancellers / total_customers) * 100

print(f"Percentage of repeat cancellers: {percentage_repeat_cancellers:.2f}%")


Percentage of repeat cancellers: 0.52%


#### 4. Do repeat cancellers generate lower revenue?

In [33]:
# =========================
# 1. BASIC CLEANING
# =========================

# Convert to numeric safely
df['Cancelled_Rides_by_Customer'] = pd.to_numeric(
    df['Cancelled_Rides_by_Customer'], errors='coerce'
)

df['Booking_Value'] = pd.to_numeric(
    df['Booking_Value'], errors='coerce'
)

# Drop rows without Customer_ID
df = df.dropna(subset=['Customer_ID'])

# Fill NaNs
df['Cancelled_Rides_by_Customer'] = df['Cancelled_Rides_by_Customer'].fillna(0)
df['Booking_Value'] = df['Booking_Value'].fillna(0)


# =========================
# 2. CUSTOMER-LEVEL METRICS
# =========================

customer_metrics = (
    df.groupby('Customer_ID')
      .agg(
          total_rides=('Booking_ID', 'count'),
          cancelled_rides=('Cancelled_Rides_by_Customer', 'sum'),
          total_revenue=('Booking_Value', 'sum')
      )
)

# =========================
# 3. CUSTOMER CLASSIFICATION
# =========================

customer_metrics['customer_type'] = customer_metrics[
    'cancelled_rides'
].apply(
    lambda x: 'Repeat Canceller' if x > 1 else 'Non-Repeat Canceller'
)

# =========================
# 4. DERIVED METRICS
# =========================

customer_metrics['avg_revenue_per_ride'] = (
    customer_metrics['total_revenue'] /
    customer_metrics['total_rides']
)

# =========================
# 5. FILTER NOISE (OPTIONAL BUT SMART)
# =========================

filtered_metrics = customer_metrics[
    customer_metrics['total_rides'] >= 5
]

# =========================
# 6. REVENUE COMPARISON
# =========================

revenue_summary = (
    filtered_metrics
    .groupby('customer_type')
    .agg(
        avg_revenue_per_customer=('total_revenue', 'mean'),
        avg_revenue_per_ride=('avg_revenue_per_ride', 'mean'),
        customer_count=('customer_type', 'count')
    )
)

print(revenue_summary)

# =========================
# 7. PERCENTAGE OF REPEAT CANCELLERS
# =========================

total_customers = customer_metrics.shape[0]
repeat_cancellers = customer_metrics[
    customer_metrics['cancelled_rides'] > 1
].shape[0]

percentage_repeat_cancellers = (
    repeat_cancellers / total_customers
) * 100

print(f"\nPercentage of repeat cancellers: {percentage_repeat_cancellers:.2f}%")


                      avg_revenue_per_customer  avg_revenue_per_ride  \
customer_type                                                          
Non-Repeat Canceller               1659.559653            319.250968   
Repeat Canceller                    912.692308            176.732051   

                      customer_count  
customer_type                         
Non-Repeat Canceller             461  
Repeat Canceller                  26  

Percentage of repeat cancellers: 0.52%


#### 5. Are repeat cancellers concentrated in specific vehicle types?


In [34]:
# -------------------------
# 1. Customer-level cancellation summary
# -------------------------
customer_cancellations = (
    df.groupby(['Customer_ID', 'Vehicle_Type'])
      .agg(
          cancelled_rides=('Cancelled_Rides_by_Customer', 'sum')
      )
      .reset_index()
)

# -------------------------
# 2. Classify repeat cancellers
# -------------------------
customer_cancellations['is_repeat_canceller'] = (
    customer_cancellations['cancelled_rides'] > 1
)

# -------------------------
# 3. Vehicle-type level concentration
# -------------------------
vehicle_concentration = (
    customer_cancellations
    .groupby('Vehicle_Type')
    .agg(
        total_customers=('Customer_ID', 'nunique'),
        repeat_cancellers=('is_repeat_canceller', 'sum')
    )
)

# -------------------------
# 4. Percentage calculation
# -------------------------
vehicle_concentration['repeat_canceller_percentage'] = (
    vehicle_concentration['repeat_cancellers'] /
    vehicle_concentration['total_customers']
) * 100

# -------------------------
# 5. Sort for insight
# -------------------------
vehicle_concentration = vehicle_concentration.sort_values(
    by='repeat_canceller_percentage',
    ascending=False
)

print(vehicle_concentration)

              total_customers  repeat_cancellers  repeat_canceller_percentage
Vehicle_Type                                                                 
Auto                    13369                 15                     0.112200
Mini                    13273                 11                     0.082875
Prime Plus              13127                  7                     0.053325
Bike                    13103                  6                     0.045791
Prime SUV               13070                  4                     0.030604
Prime Sedan             13251                  4                     0.030186
eBike                   13273                  2                     0.015068


#### 6. Are repeat cancellers more active on weekends?

In [39]:

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

customer_cancel_df = (
    df.loc[df['Reason_for_cancelling_by_Customer'].notna()]
    .copy()
)

customer_cancel_df.loc[:, 'cancel_count'] = (
    customer_cancel_df
    .groupby('Customer_ID')['Booking_ID']
    .transform('count')
)

repeat_df = customer_cancel_df.loc[
    customer_cancel_df['cancel_count'] > 1
].copy()

if repeat_df.empty:
    print("Conclusion: No repeat cancellers found in the dataset.")
else:
    repeat_df.loc[:, 'day_number'] = repeat_df['Date'].dt.dayofweek
    repeat_df.loc[:, 'is_weekend'] = repeat_df['day_number'] >= 5

    summary = (
        repeat_df
        .groupby('is_weekend')
        .agg(
            cancel_events=('Booking_ID', 'count'),
            unique_customers=('Customer_ID', 'nunique')
        )
        .reset_index()
    )

    total_events = summary['cancel_events'].sum()
    summary.loc[:, 'event_share'] = summary['cancel_events'] / total_events

    print(summary)

    weekend_share = summary.loc[summary['is_weekend'], 'event_share']
    weekday_share = summary.loc[~summary['is_weekend'], 'event_share']

    if (
        not weekend_share.empty and
        not weekday_share.empty and
        weekend_share.iloc[0] > weekday_share.iloc[0]
    ):
        print("Conclusion: Repeat cancellers are more active on weekends.")
    else:
        print("Conclusion: Repeat cancellers are NOT more active on weekends.")


   is_weekend  cancel_events  unique_customers  event_share
0       False            425               278     0.658915
1        True            220               178     0.341085
Conclusion: Repeat cancellers are NOT more active on weekends.


#### 7. Can customers be segmented into low, medium, and high risk?

In [42]:

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

customer_risk = (
    df.groupby('Customer_ID')
    .agg(
        total_rides=('Booking_ID', 'count'),
        customer_cancellations=('Reason_for_cancelling_by_Customer', 'count'),
        incomplete_rides=('Incomplete_Rides', 'sum')
    )
    .reset_index()
)

customer_risk['risk_score'] = (
    customer_risk['customer_cancellations'] +
    customer_risk['incomplete_rides']
)

customer_risk['risk_rank_pct'] = (
    customer_risk['risk_score']
    .rank(method='average', pct=True)
)

customer_risk['risk_segment'] = pd.cut(
    customer_risk['risk_rank_pct'],
    bins=[0, 0.33, 0.66, 1.0],
    labels=['Low Risk', 'Medium Risk', 'High Risk'],
    include_lowest=True
)

segment_summary = (
    customer_risk
    .groupby('risk_segment',observed=True)
    .agg(
        customers=('Customer_ID', 'count'),
        avg_risk_score=('risk_score', 'mean'),
        max_risk_score=('risk_score', 'max'),
        avg_rides=('total_rides', 'mean')
    )
    .reset_index()
)

print(segment_summary)


  risk_segment  customers  avg_risk_score  max_risk_score  avg_rides
0  Medium Risk      48451        0.000000             0.0   1.560835
1    High Risk      12039        1.076252             4.0   2.024753


#### 8. What proportion of revenue comes from high-risk customers?

In [43]:
df['Booking_Value'] = pd.to_numeric(df['Booking_Value'], errors='coerce')

customer_revenue = (
    df.groupby('Customer_ID')
    .agg(
        total_revenue=('Booking_Value', 'sum')
    )
    .reset_index()
)

customer_risk_revenue = customer_risk.merge(
    customer_revenue,
    on='Customer_ID',
    how='left'
)

revenue_summary = (
    customer_risk_revenue
    .groupby('risk_segment', observed=True)
    .agg(
        customers=('Customer_ID', 'count'),
        total_revenue=('total_revenue', 'sum'),
        avg_revenue_per_customer=('total_revenue', 'mean')
    )
    .reset_index()
)

total_revenue_all = revenue_summary['total_revenue'].sum()

revenue_summary['revenue_share'] = (
    revenue_summary['total_revenue'] / total_revenue_all
)

print(revenue_summary)

high_risk_share = revenue_summary.loc[
    revenue_summary['risk_segment'] == 'High Risk',
    'revenue_share'
].values

if len(high_risk_share) > 0:
    print(
        f"Proportion of revenue from high-risk customers: "
        f"{high_risk_share[0]:.2%}"
    )
else:
    print("No high-risk customers found.")


  risk_segment  customers  total_revenue  avg_revenue_per_customer  \
0  Medium Risk      48451     26333789.0                543.513839   
1    High Risk      12039      3941778.0                327.417393   

   revenue_share  
0       0.869803  
1       0.130197  
Proportion of revenue from high-risk customers: 13.02%


### 🔍 F. Operational & Business Insights

#### 1. Which factors most strongly influence successful booking completion?

- Successful booking completion is primarily influenced by customer cancellation history, vehicle-type reliability, and time-related factors such as weekday versus weekend demand. 
Customers with fewer past cancellations and bookings made during weekdays tend to complete rides more consistently. 
Driver-related factors, particularly higher driver ratings, are also associated with higher completion rates. 

#### 2. Which vehicle types are most operationally stable?

- Vehicle of Mini and Auto has most operationally stable 

#### 3. Which cancellation reason has the highest operational impact?

- Cancellation reasons related to driver availability and customer no-shows have the highest operational impact, as they directly lead to lost revenue, 
wasted driver time, and reduced platform efficiency.

#### 4. Are driver cancellations more frequent for specific vehicle types?

- Yes, driver cancellations tend to be more frequent for certain vehicle types, particularly those with higher demand volatility or operational complexity.
This suggests that vehicle-specific constraints influence driver reliability

#### 5. Are incomplete rides clustered around certain dates or patterns?

- Incomplete rides show clustering around specific periods, such as high-demand days and peak time windows.
This indicates that operational pressure during busy periods contributes to ride incompletion.

#### 6. Which payment methods are least associated with cancellations?

- Digital and prepaid payment methods—such as UPI, Credit Card, Debit Card, and Ola Money—are least associated with cancellations when compared to cash payments.
These methodms indicate higher customer commitment and smoother transaction flow, as payment intent is established upfront. 
In contrast, cash payments show relatively higher cancellation association, likely due to lower commitment and flexibility to cancel without financial friction.

#### 7. Does a higher driver rating reduce cancellation likelihood?

- Yes, higher driver ratings are associated with lower cancellation likelihood. Better-rated drivers tend to provide more reliable service,
resulting in fewer cancellations and incomplete rides.

#### 8. Which engineered features should be prioritised for dashboard KPIs?

- The most important engineered features for dashboard KPIs include:
            
            1. Booking completion rate
            
            2. Customer cancellation rate
            
            3. Driver cancellation rate
            
            4. Incomplete ride rate
            
            5. Revenue per booking
            
            6. High-risk customer share
            
These metrics directly reflect operational health and customer behavior.


### 🧱 G. Analysis Code Skeleton (Conceptual Questions)

#### 1. What columns should be selected for Pandas analysis?

- The Following Columns should be taken into consideration for pandas analysis:
    1. Cancelled_Rides_by_Customer
    2. Cancelled_Rides_by_Driver
    3. Vehicle_Type
    4. Booking_Status
    5. Booking_Value
    6. Payment_Method
    7. Ride_Distance
    8. Driver_Ratings
    9. Customer_Rating

#### 2. What transformations should be applied before analysis?

- Before tranformation to be applied, following thing are needed to be considered:
1. Drop rows with >50% nulls
2. Drop columns only if they are mostly null and non-critical
3. Filling the missing values with help of imputation techniques.
4. Ensure each columns have precise data type according data contain in that column.


#### 3. What grouping levels are needed (booking-level vs customer-level)?

- Grouping are needed to be done of following levels:
    1. Booking Status- it showcases booking level in 4 different categories
    2. Customer-level aggregation reveals satisfaction trends and loyalty risk


#### 4. What aggregations are required before visualisation?


- Aggregations required before visualisation

- Before visualisation, the following aggregations are required to ensure clarity and meaningful insights:

    1. Booking-level aggregations
       - Counts and percentages of completed, cancelled, and incomplete bookings.
    
    2. Customer-level aggregations
        - Total rides per customer, total cancellations, cancellation rate, and risk segment counts.
    
    3. Revenue aggregations
       - Total and average booking value by vehicle type, payment method, and customer segment.
    
    4. Time-based aggregations
       - Booking counts and cancellation rates grouped by day of week (weekday vs weekend) and date.
    
    5. Vehicle-type aggregations
        - Average ride distance, average booking value, and cancellation rate per vehicle type.
    
    6. Rating aggregations
        - Average driver and customer ratings by booking status and vehicle type.
    
    7. Payment-method aggregations
        - Cancellation rate and average booking value grouped by payment method.


#### 5. Which metrics should be precomputed instead of calculated in visuals?


- Metrics that should be precomputed before visualisation

- The following metrics should be precomputed to ensure consistency, performance, and clarity in visualisations:

    1. Booking completion rate
        - Ratio of completed bookings to total bookings.

    2. Cancellation rate
        - Cancellations divided by total bookings, at booking, customer, or vehicle level.

    3. Customer-level cancellation rate
        - Total cancellations per customer divided by total rides.

    4. Risk score and risk segment
        - Derived customer risk metrics based on cancellation and incomplete ride behavior.

    5. Revenue per booking
        - Average booking value by vehicle type, payment method, or customer segment.

    6. Revenue share by segment
        - Proportion of total revenue contributed by each customer or risk segment.

    7. Incomplete ride rate
        - Incomplete rides divided by total bookings.

    8. Average ratings by category
        - Pre-aggregated average driver and customer ratings by booking status or vehicle type.

    9. Weekend vs weekday indicators
        - Binary flags and aggregated rates based on time-based classification.

#### 6. What statistical measures should be computed using NumPy?

- Statistical measures that should be computed using NumPy

- The following statistical measures should be computed using NumPy to understand data distribution, variability, and relationships:

    1.  Central tendency measures
        - Mean and median of booking values, ride distances, and ratings.

    2. Dispersion measures
        - Standard deviation, variance, and range for booking values and ride distances.

    3. Percentiles and quantiles
        - 25th, 50th, 75th percentiles to analyze skewness and outliers in revenue and distance.

    4. Correlation coefficients
        - Correlation between booking value and ride distance, and between ratings and cancellation behavior.

    5. Z-scores and standardization
        - Identification of extreme values or anomalies in booking value and ride distance.

    6. Distribution shape indicators
        - Skewness and kurtosis to assess whether metrics are normally distributed or heavy-tailed.

    7. Comparative statistics
        - Mean and variance comparison between completed and cancelled bookings.

#### 7. What derived columns should be validated against SQL outputs?

- Derived columns that should be validated against SQL outputs

- The following derived columns should be validated against SQL results to ensure consistency and correctness across analysis layers:

    1. Total bookings per customer
        - Validate customer-level ride counts.

    2. Total cancellations (customer and driver)
        - Ensure cancellation counts match SQL aggregations.

    3. Booking completion flag and completion rate
        - Confirm that completed vs cancelled/incomplete classification aligns with SQL logic.

    4. Total revenue and average booking value
        - Cross-check revenue calculations and averages.

    5. Customer cancellation rate
        - Verify ratios of cancellations to total rides.

    6. Weekend vs weekday classification
        - Ensure date-based flags are derived identically in Pandas and SQL.

    7. Risk score and risk segment
        - Validate customer risk metrics and segmentation logic.

    8. Incomplete ride counts and rates
        - Confirm incomplete ride aggregations.

    9. Average ratings (driver and customer)
        - Validate rating aggregations by booking status or vehicle type.

#### 8. What assumptions must be documented before visualisation?

- Assumptions that must be documented before visualisation

- Before visualisation, the following assumptions should be clearly documented to ensure correct interpretation of insights:

    1. Definition of booking success and cancellation
        - Assumes booking status labels consistently represent completed, cancelled, and incomplete rides.

    2. Customer and driver behavior consistency
        - Assumes past customer and driver behavior is indicative of future patterns.

    3. Handling of missing and NULL values
        - Assumes missing ratings, distances, or booking values are either excluded or handled consistently.

    4. Revenue calculation logic
        - Assumes booking value represents actual realized revenue and excludes cancelled or incomplete rides where applicable.

    5. Risk segmentation logic
        - Assumes customer risk categories are derived solely from historical cancellation and incomplete ride behavior.

    6. Time-based classification
        - Assumes date and time fields are accurate and that weekday/weekend classification reflects true operational patterns.

    7. Equal weighting of bookings
        - Assumes all bookings contribute equally unless explicitly weighted by revenue or distance.

    8. No external factors included
        - Assumes external influences (weather, traffic, promotions) are not accounted for in the analysis.

#### 9. Which insights are strong enough to justify a chart?

- The following insights are strong, interpretable, and decision-relevant, making them suitable for visualisation:

    1. Booking status distribution (Completed vs Cancelled vs Incomplete)
        - Provides an immediate overview of operational health.

    2. Cancellation rate by vehicle type
        - Clearly highlights operational stability and problem areas.

    3. Revenue contribution by customer risk segment
        - Shows whether high-risk customers are financially significant.

    4. Booking value distribution
        - Reveals skewness, outliers, and typical transaction ranges.

    5. Ride distance vs booking value relationship
        - Helps assess pricing logic and distance sensitivity.

    6. Cancellation rate: weekday vs weekend
        - Identifies time-based operational stress patterns.

    7. Average ratings by booking status or vehicle type
        - Connects service quality to outcomes.

    8. Payment method vs cancellation rate
        - Highlights behavioral differences linked to payment commitment.

#### 10. Which insights should be excluded from visualisation due to low impact? 

- Insights that should be excluded from visualisation due to low impact

- The following insights should generally be excluded from visualisation because they add limited business value or do not meaningfully influence decisions:

    1. Extremely rare categories
        - Vehicle types, payment methods, or cancellation reasons with very low frequency, as they do not represent actionable patterns.

    2. Metrics with minimal variation
        - Insights where values are nearly constant across categories, such as uniformly high or low ratings.

    3. Redundant metrics
        - Metrics that convey the same information as another chart (e.g., total bookings and total completed bookings shown separately).

    4. Highly granular time breakdowns
        - Hour-by-hour or date-level views when no clear trend or seasonality is present.

    5. Low-impact correlations
        - Weak relationships between variables that do not materially affect outcomes or decisions.

    6. Derived metrics with negligible differences
        - Segments where performance differences are statistically or practically insignificant.

    7. Outlier-driven insights
        - Observations based on a very small number of extreme cases that do not reflect typical behavior.